<a href="https://colab.research.google.com/github/shaalu2408-pixel/python-module-5/blob/main/Python_DA_Assignment_3_Web_Scraping_and_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Web Scraping**

# 1. Extract Book Data from Website


In [ ]:
!pip install requests
!pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
url= 'https://books.toscrape.com/'
headers={
    "User-Agent": "Mozilla/5.0 (windows NT 10.0; Win64; x64)"
                   "Applewebkit/537.36(KHTML, like Gecko)"
                   "Chromr/120.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [ ]:
html_code=response.text
html_code

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" href

In [ ]:
soup = bs(html_code,'html.parser')
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

# **1.Scrape the following details for each book on the page**

# **Book Title**

In [ ]:
soup.title.text

'\n    All products | Books to Scrape - Sandbox\n'

# **Price**

In [ ]:
prices=soup.find_all('p',class_='price_color')
for price in prices:
    print(price.text)

Â£51.77
Â£53.74
Â£50.10
Â£47.82
Â£54.23
Â£22.65
Â£33.34
Â£17.93
Â£22.60
Â£52.15
Â£13.99
Â£20.66
Â£17.46
Â£52.29
Â£35.02
Â£57.25
Â£23.88
Â£37.59
Â£51.33
Â£45.17


# **Rating**

In [ ]:
ratings=[]
books= soup.find_all('article',class_='product_pod')
for book in books:
  rating=book.p['class'][1]
  ratings.append(rating)
print("Ratings:",ratings[:10])

Ratings: ['Three', 'One', 'One', 'Four', 'Five', 'One', 'Four', 'Three', 'Four', 'One']


# **Availability (In stock/Out of stock)**

In [ ]:
availability=[]
books= soup.find_all('article',class_='product_pod')
for book in books:
  avail=book.find('p',class_='instock availability').text.strip()
  availability.append(avail)
print("Availability:",availability[:5])


Availability: ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']


# **Genre**

In [ ]:
base_url='https://books.toscrape.com/catalogue/'
page_url='https://books.toscrape.com/catalogue/page-1.html'
response=requests.get(page_url)
soup=bs(response.text,'html.parser')
books=soup.find_all('article',class_='product_pod')
genres=[]
for book in books:
  link=book.find('h3').find('a')['href']
  book_url=base_url+link
  book_page=requests.get(book_url)
  book_soup=bs(book_page.text,'html.parser')
  genre=book_soup.find('ul',class_='breadcrumb').find_all('a')[2].text
  genres.append(genre)
print(genres)

['Poetry', 'Historical Fiction', 'Fiction', 'Mystery', 'History', 'Young Adult', 'Business', 'Default', 'Default', 'Poetry', 'Default', 'Poetry', 'Young Adult', 'Sequential Art', 'Music', 'Music', 'Poetry', 'Science Fiction', 'Politics', 'Travel']


# **2. Pagination Handling**

In [ ]:
import pandas as pd
import requests
from urllib.parse import urljoin
from time import sleep

base_url='https://books.toscrape.com/catalogue/page-1.html'
titles=[]
prices=[]
ratings=[]
availability=[]
genres=[]

for page in range(1,51):
  url=base_url.format(page)
  response=requests.get(url)
  soup=bs(response.text,'html.parser')
  books=soup.find_all('article',class_='product_pod')

  for book in books:
    title= book.h3.a['title']
    titles.append(title)

    price = book.find('p', class_='price_color').text.strip()
    prices.append(price)

    rating=book.p['class'][1]
    ratings.append(rating)

    avail=book.find('p',class_='instock availability').text.strip()
    availability.append(avail)

    book_link = book.h3.a['href']
    full_url = urljoin('https://books.toscrape.com/catalogue/',book_link)
    detail_page = requests.get(full_url)
    detail_soup = bs(book_page.text,'html.parser')

    breadcrumb = detail_soup.select('ul.breadcrumb li a')
    if len(breadcrumb) >= 3:
      genre = breadcrumb[2].text.strip()
    else:
      genre = 'Unknown'
    genres.append(genre)
    sleep(0.2)

  print('Total books scraped:',len(titles))
  print('first 5 genres[:5]',genres[:5])








Total books scraped: 20
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 40
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 60
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 80
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 100
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 120
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 140
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 160
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 180
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 200
first 5 genres[:5] ['Travel', 'Travel', 'Travel', 'Travel', 'Travel']
Total books scraped: 220
first 5 genres[:5] ['Travel',

# **3. Store Data in a CSV File**

In [ ]:
import pandas as pd
min_len=min(len(titles),len(prices),len(ratings),len(availability),len(genres))
titles=titles[:min_len]
prices=prices[:min_len]
ratings=ratings[:min_len]
availability=availability[:min_len]
genres=genres[:min_len]
df=pd.DataFrame({
    'Title':titles,
    'Price':prices,
    'Ratings':ratings,
    'Availability':availability,
    'Genre':genres
})
df.to_csv("books_data.csv",index=0)
print("Data scraping complete!")
print("data saved as: books_data.csv")
print(df.head())


Data scraping complete!
data saved as: books_data.csv
                                   Title    Price Ratings Availability   Genre
0                   A Light in the Attic  Â£51.77   Three     In stock  Travel
1                     Tipping the Velvet  Â£53.74     One     In stock  Travel
2                             Soumission  Â£50.10     One     In stock  Travel
3                          Sharp Objects  Â£47.82    Four     In stock  Travel
4  Sapiens: A Brief History of Humankind  Â£54.23    Five     In stock  Travel


# **PySpark**

# **1. Setup PySpark Environment**

In [ ]:
!apt-get install openjdk-11-jdk -y
!pip install pyspark

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.28+6-1ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
pyspark.__version__

'3.5.1'

In [ ]:
spark = SparkSession.builder.appName('BooksFilter').getOrCreate()

# **2. Create a Spark DataFrame**

In [ ]:
df = spark.read.csv("/content/books_data.csv",header=True,inferSchema=True)
df

DataFrame[Title: string, Price: string, Ratings: string, Availability: string, Genre: string]

# **Checking the schema of the DataFrame**.

In [ ]:
df.schema

StructType([StructField('Title', StringType(), True), StructField('Price', StringType(), True), StructField('Ratings', StringType(), True), StructField('Availability', StringType(), True), StructField('Genre', StringType(), True)])

# **Viewing the first few rows**

In [ ]:
df.show()

+--------------------+-------+-------+------------+------+
|               Title|  Price|Ratings|Availability| Genre|
+--------------------+-------+-------+------------+------+
|A Light in the Attic|Â£51.77|  Three|    In stock|Travel|
|  Tipping the Velvet|Â£53.74|    One|    In stock|Travel|
|          Soumission|Â£50.10|    One|    In stock|Travel|
|       Sharp Objects|Â£47.82|   Four|    In stock|Travel|
|Sapiens: A Brief ...|Â£54.23|   Five|    In stock|Travel|
|     The Requiem Red|Â£22.65|    One|    In stock|Travel|
|The Dirty Little ...|Â£33.34|   Four|    In stock|Travel|
|The Coming Woman:...|Â£17.93|  Three|    In stock|Travel|
|The Boys in the B...|Â£22.60|   Four|    In stock|Travel|
|     The Black Maria|Â£52.15|    One|    In stock|Travel|
|Starving Hearts (...|Â£13.99|    Two|    In stock|Travel|
|Shakespeare's Son...|Â£20.66|   Four|    In stock|Travel|
|         Set Me Free|Â£17.46|   Five|    In stock|Travel|
|Scott Pilgrim's P...|Â£52.29|   Five|    In stock|Trave

# **Showing the summary statistics for numerical columns**

In [ ]:
df.describe().show()

+-------+--------------------+-------+-------+------------+------+
|summary|               Title|  Price|Ratings|Availability| Genre|
+-------+--------------------+-------+-------+------------+------+
|  count|                1000|   1000|   1000|        1000|  1000|
|   mean|                NULL|   NULL|   NULL|        NULL|  NULL|
| stddev|                NULL|   NULL|   NULL|        NULL|  NULL|
|    min|A Light in the Attic|Â£13.99|   Five|    In stock|Travel|
|    max|  Tipping the Velvet|Â£57.25|    Two|    In stock|Travel|
+-------+--------------------+-------+-------+------------+------+



# **3. Filtering Data**

Apply a filter to the DataFrame based on a condition.  
(For example, filter the books with a price greater than a certain amount
like $20 or books with a certain rating like 4 stars and above).

In [ ]:
df_raw = spark.read.csv("books_data.csv", header=True, inferSchema=False)
df_raw.select("Ratings", "Price").show(20, truncate=False)
from pyspark.sql.functions import when, lower, trim, col, regexp_replace

+-------+-------+
|Ratings|Price  |
+-------+-------+
|Three  |Â£51.77|
|One    |Â£53.74|
|One    |Â£50.10|
|Four   |Â£47.82|
|Five   |Â£54.23|
|One    |Â£22.65|
|Four   |Â£33.34|
|Three  |Â£17.93|
|Four   |Â£22.60|
|One    |Â£52.15|
|Two    |Â£13.99|
|Four   |Â£20.66|
|Five   |Â£17.46|
|Five   |Â£52.29|
|Five   |Â£35.02|
|Three  |Â£57.25|
|One    |Â£23.88|
|One    |Â£37.59|
|Two    |Â£51.33|
|Two    |Â£45.17|
+-------+-------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import regexp_replace, trim, col, when
df_clean = df_raw.withColumn(
    "Price",
    regexp_replace(trim(col("Price")), "Â£|£", "").cast("double")
)
df_clean = df_clean.withColumn(
    "RatingsNum",
    when(lower(col("Ratings")) == "one", 1)
    .when(lower(col("Ratings")) == "two", 2)
    .when(lower(col("Ratings")) == "three", 3)
    .when(lower(col("Ratings")) == "four", 4)
    .when(lower(col("Ratings")) == "five", 5)
)

In [ ]:
df_filtered = df_clean.filter((col("Price") > 20) | (col("RatingsNum") >= 4))    #filter the book > 20 with price value 'OR' rating having >= 4
df_filtered.select("Ratings", "Price").show(20, truncate=False)


+-------+-----+
|Ratings|Price|
+-------+-----+
|Three  |51.77|
|One    |53.74|
|One    |50.1 |
|Four   |47.82|
|Five   |54.23|
|One    |22.65|
|Four   |33.34|
|Four   |22.6 |
|One    |52.15|
|Four   |20.66|
|Five   |17.46|
|Five   |52.29|
|Five   |35.02|
|Three  |57.25|
|One    |23.88|
|One    |37.59|
|Two    |51.33|
|Two    |45.17|
|Three  |51.77|
|One    |53.74|
+-------+-----+
only showing top 20 rows

